In [0]:
# importing required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install yfinance
import yfinance as yf
!pip install yahoofinancials
import yahoofinancials

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=258fabd9fc284c608a73fe261b42f553304f8e4de7b9fb1a11b4a36339293b12
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance
  Created wheel for yahoofinancials: filename=yahoofinancials-1.5-cp36-none-any.whl size=15195 sha256=75c8f5ed9306a9fdc0d543d6f89fd02189657e32fd7e20562dca658aa1e3d9f5
  Stored in directory: /root/.cache/pip/wheels/42/15/e7/240c5cdc8782b9479ba73404640a3be57bb4053a5c3edd6be5
Successfully built yahoofinancials


In [0]:
# name of major stock markets 
stock_name = ["NYSE COMPOSITE (DJ)","NASDAQ Composite","Japan Exchange Group, Inc.","London Stock Exchange Group plc","Shanghai Stock Exchange","Hong Kong Exchanges","S&P BSE SENSEX","NIFTY 50","SSE Composite Index"]

In [0]:
# code of major stock markets 
stock_code = ["^NYA","^IXIC","JPXGY","LSE.L","^SSEC","0388.HK","^BSESN","^NSEI","000001.SS" ]

In [0]:
# curency conversion rate for stock market data 
currency = ["USD","USD","USD","GBp","CNY","HKD","INR","INR","CNY"]

In [0]:
data = pd.DataFrame(columns=["stock_name","stock_code","currency"]) 

In [0]:
data.stock_code = stock_code
data.stock_name = stock_name
data.currency = currency

In [0]:
data

,stock_name,stock_code,currency
0,NYSE COMPOSITE (DJ),^NYA,USD
1,NASDAQ Composite,^IXIC,USD
2,"Japan Exchange Group, Inc.",JPXGY,USD
3,London Stock Exchange Group plc,LSE.L,GBp
4,Shanghai Stock Exchange,^SSEC,CNY
5,Hong Kong Exchanges,0388.HK,HKD
6,S&P BSE SENSEX,^BSESN,INR
7,NIFTY 50,^NSEI,INR
8,SSE Composite Index,000001.SS,CNY


In [0]:
# loop to store the converted currencies and store them in a stacked fashion
k = 0 # for row iteration
tableau_data = pd.DataFrame(columns=["Date","Close","Name"])                                                      # empty dataframe
for i in range(data['stock_code'].shape[0]):
  temp_data = yf.download(data.loc[i,"stock_code"], start='2019-11-01', end='2020-04-10', progress=False)         # download dataset from yahoo financials
  if data.loc[i,"currency"] == "GBp":                                                                             # currency conversion
    temp_data["Close"] = 1.25*temp_data["Close"]
  elif data.loc[i,"currency"] == "CNY":
    temp_data["Close"] = 0.14*temp_data["Close"]
  elif data.loc[i,"currency"] == "HKD":
    temp_data["Close"] = 0.13 *temp_data["Close"]
  elif data.loc[i,"currency"] == "INR":
    temp_data["Close"] = 0.013 *temp_data["Close"]
    
  if temp_data.any != None:                                                                                         # check for null values
    temp_data['Date'] = temp_data.index
    temp_data.reset_index(drop=True,inplace= True)
    temp_data = temp_data[["Date","Close"]]
    temp_data["Name"] = data.loc[i,"stock_name"]
    
    for index, row in temp_data.iterrows():                                                                        # loop to store it in a stacked fashion 
  
      tableau_data.loc[index+k,"Date"] = row['Date']
      tableau_data.loc[index+k,"Close"] = row['Close']
      tableau_data.loc[index+k,"Name"] = row['Name']
      k=k+index
  else:
    print("not found")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1 Failed download:
- ^SSEC: No data found for this date range, symbol may be delisted


In [0]:
# saving the file to a csv format
tableau_data.to_csv("tableau_data.csv")

In [0]:
tableau_data

,Date,Close,Name
0,2019-11-01 00:00:00,13300.3,NYSE COMPOSITE (DJ)
1,2019-11-04 00:00:00,13355.4,NYSE COMPOSITE (DJ)
3,2019-11-05 00:00:00,13339.6,NYSE COMPOSITE (DJ)
6,2019-11-06 00:00:00,13351.6,NYSE COMPOSITE (DJ)
10,2019-11-07 00:00:00,13395.5,NYSE COMPOSITE (DJ)
...,...,...,...
47011,2020-04-02 00:00:00,389.289,SSE Composite Index
47113,2020-04-03 00:00:00,386.958,SSE Composite Index
47216,2020-04-07 00:00:00,394.907,SSE Composite Index
47320,2020-04-08 00:00:00,394.152,SSE Composite Index


In [0]:
# scraping data from world-o-meter for coronavirus cases

In [0]:
res = requests.get("https://www.worldometers.info/coronavirus/")                # using a get request method to extract the html file
soup = BeautifulSoup(res.content,'lxml')                                        # using beautiful soup to extract the information from html file
table = soup.find_all('table')[1] 
df = pd.read_html(str(table))[0]                                                # changing it to pandas dataframe.


In [0]:
df.to_csv("current_cases.csv")